In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

In [3]:
train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


NameError: name 'train_x' is not defined

In [ ]:
train_x=train.drop(['Survived'],axis=1)
train_y=train['Survived']
test_x=test.copy()

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
train_x = train_x.drop(['PassengerId'],axis=1)
test_x = test_x.drop(['PassengerId'],axis=1)

In [ ]:
train_x2=train_x
test_x2=train_x

In [ ]:
train_x2=train_x
test_x2=test_x

In [ ]:
train_x2

In [ ]:
test_x2

In [ ]:
for c in ['Sex','Embarked']:
    le = LabelEncoder()
    le.fit(train_x[c].fillna('NA'))
    train_x[c]=le.transform(train_x[c].fillna('NA'))
    test_x[c]=le.transform(test_x[c].fillna('NA'))

In [ ]:
model = X

In [ ]:
from xgboost import XGBClassifier

In [ ]:
model = XGBClassifier(n_estimators=20,ramdom_state=71)
model.fit(train_x,train_y)

In [ ]:
pred = model.predict_proba(test_x)[:,1]
pred_label = np.where(pred > 0.5,1,0)

In [ ]:
submission = pd.DataFrame({'PassengerId':test['PassengerId'],'Survived':pred_label})
submission.to_csv('submission_first.csv',index=False)

In [ ]:
from sklearn.metrics import log_loss, accuracy_score
from sklearn.model_selection import KFold

In [ ]:
scores_accuracy=[]
scores_logloss=[]

In [ ]:
kf = KFold(n_splits=4,shuffle=True,random_state=71)
for tr_idx, va_idx in kf.split(train_x):
    tr_x,va_x = train_x.iloc[tr_idx],train_x.iloc[va_idx]
    tr_y,va_y = train_y.iloc[tr_idx],train_y.iloc[va_idx]
    
    model = XGBClassifier(n_estimators=20,ramdom_state=71)
    model.fit(tr_x,tr_y)
    
    va_pred = model.predict_proba(va_x)[:,1]
    
    logloss = log_loss(va_y,va_pred)
    accuracy = accuracy_score(va_y,va_pred > 0.5)
    
    scores_logloss.append(logloss)
    scores_accuracy.append(accuracy)
    
logloss = np.mean(scores_logloss)
accuracy - np.mean(scores_accuracy)
print(f'logloss:{logloss:.4f}, accuracy:{accuracy:.4f}')

In [ ]:
import itertools

In [ ]:
param_space = {
    'max_depth' : [3,5,7],
    'min_child_weight':[1.0,2.0,4.0]
}

param_combinations = itertools.product(param_space['max_depth'],param_space["min_child_weight"])

params = []
scores = []

for max_depth, min_child_weight in param_combinations:
    
    score_folds=[]
    
    kf = KFold(n_splits=4,shuffle=True,random_state=123456)
    for tr_idx, va_idx,in kf.split(train_x):
        tr_x,va_x=train_x.iloc[tr_idx],train_x.iloc[va_idx]
        tr_y,va_y=train_y.iloc[tr_idx],train_y.iloc[va_idx]

        model = XGBClassifier(n_estimators=20,random_state=71,max_depth=max_depth,min_child_weight=min_child_weight)

        model.fit(tr_x,tr_y)

        va_pred = model.predict_proba(va_x)[:,1]
        logloss = log_loss(va_y, va_pred)
        score_folds.append(logloss)
        
    score_mean = np.mean(score_folds)
    
    params.append((max_depth,min_child_weight))
    scores.append(score_mean)
    
best_idx = np.argsort(scores)[0]
best_param = params[best_idx]
print(f'max_depth:{best_param[0]},min_child_weight:{best_param[1]}')



In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder

In [ ]:
train_x2 = train.drop(['Survived'],axis=1)
test_x2=test.copy()

train_x2=train_x2.drop(['PassengerId'],axis=1)
test_x2=test_x2.drop(['PassengerId'],axis=1)

train_x2 = train_x2.drop(['Name','Ticket','Cabin'],axis=1)
test_x2 = test_x2.drop(['Name','Ticket','Cabin'],axis=1)

In [ ]:
cat_cols=['Sex','Embarked','Pclass']
one = OneHotEncoder(categories='auto',sparse=False)
one.fit(train_x2[cat_cols].fillna('NA'))

In [ ]:
one_columns=[]
for i,c in enumerate(cat_cols):
    one_columns += [f'{c}_{v}' for v in one.categories_[i]]

In [ ]:
one_train_x2 = pd.DataFrame(one.transform(train_x2[cat_cols].fillna('NA')),columns=one_columns)
one_test_x2 = pd.DataFrame(one.transform(test_x2[cat_cols].fillna('NA')),columns=one_columns)

In [ ]:
train_x2 = train_x2.drop(cat_cols,axis=1)
test_x2 = test_x2.drop(cat_cols,axis=1)

In [ ]:
train_x2 = pd.concat([train_x2,one_train_x2],axis=1)
test_x2 = pd.concat([test_x2,one_test_x2],axis=1)

In [ ]:
num_cols = ['Age','SibSp','Parch','Fare']
for col in num_cols:
    train_x2[col].fillna(train_x2[col].mean(),inplace=True)
    test_x2[col].fillna(train_x2[col].mean(),inplace=True)

In [ ]:
train_x2['Fare'] = np.log1p(train_x2['Fare'])
test_x2['Fare'] = np.log1p(test_x2['Fare'])